In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from urllib import request
from urllib.error import HTTPError, URLError
from bs4 import BeautifulSoup
import re
import sys
import json

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

ModuleNotFoundError: No module named 'selenium'

In [ ]:
# """
# 起動1
# java -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role hub

# 起動2
# java -Dwebdriver.chrome.driver="C:\Users\mmi-lab\Documents\selenium\chromedriver.exe" -jar C:\Users\mmi-lab\Documents\selenium\selenium-server-standalone-3.141.59.jar -role node -hub http://192.168.1.224:4444

# """

# """
# 使えそう
# ジャンルからフレーズ・例文を探す
# https://gogakuru.com/english/phrase/genre/index.html

# """
print("aa")

aa


In [ ]:
def is_declinable(token):
    pos = token.tag_.split("-")
    try:
        if pos[0] == "動詞" or pos[0] == "形容詞" or pos[2] == "サ変可能":
            # print(token.lemma_)
            return True
        else:
            False
    except:
        return False

In [ ]:

def NER(t):
    doc = nlp(t)
    for ent in doc.ents:
        print(ent)

In [ ]:
def depend_extract(t): 
    doc = nlp(t)
    for sent in doc.sents:
        for token in sent:
            # if is_declinable(token):
            if token.pos_ == "VERB":
                    # print()
                    # register_pred_count(pred_cound_dict, token.lemma_)
                for c in token.children:
                        # print(c)
                    for c2 in c.children:
                        if c2.dep_ == "case":
        
                            print( c.lemma_, c2, token.lemma_)
                            # register_pred_case_lite(pred_case_count_dict, token.lemma_, c2.text, c)
                            # print()

In [ ]:
path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

In [ ]:
# path = "../hand_labeled/"
# datalist = ['DCM', 'DIT', 'IRS']
# convs = read_conv(path, datalist)

In [9]:
t = "インターネットは排出量が多い"
t = "マドリッドの原宿、表参道エリアを中心に人気専門店が続々登場するなど、今年一躍話題になったポップコーンですが、日本穀物検定協会が金曜ロードSHOW!とコラボレーションしたきょうは会社休みます。を12月に発売するそうです。"
# # t = "固有抽出表現は、文から日付や地名等の固有表現を抽出する処理です。"
# t = "ペトリコールは雨ですね"
# t  ="旅色というスペインの赤ワインや、プレーンズ・トレインズ・アンド・エリック〜ジャパン・ツアー 2014というマドリッドの白ワインはとても甘口なのでワインが苦手な方にもおすすめです。"
# t = "フランスの画家宇佐美貴史の油絵「牧場しぼり
# 」が、ボルドーで競売に掛けられ、予想額を大幅に上回る約７５億円で落札されました。"
# t = "腰が治りますよねー"
t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"

In [ ]:
depend_extract(t)

In [8]:
# doc = nlp(t)
# for noun in doc.noun_chunks:
#     print(noun)
# make_bunsetu_ngram_suf(t)

In [10]:
from spacy import displacy
displacy.render(nlp(t), style='dep', jupyter=True, options={'compact':True, 'distance': 90})

In [ ]:
doc = nlp(t)
text = ""
for span in doc.noun_chunks:
    noun = span.text
    print(span)
    # for left in span.rights:
    #     print("\t", left)
    #     noun += left.text
    #     r = left
    #     for rr in r.rights:
    #     #    if rr.pos_
    #         print("\t\t", rr)
    #         r = rr
    #         noun += rr.text
            
    #         # noun -= rr.text
    #     print(noun)
    #     text += noun
    #     # リセット
    #     # noun = span.text
    # print()
    # print(text)

旅色
スペイン
赤ワイン
プレーンズ・トレインズ・アンド・エリック
マドリッド
白ワイン
ワイン
苦手な方
おすすめ


In [12]:
import ginza
def to_dependency_data(doc):
    words = []
    arcs = []
    bunsetu_head_list = ginza.bunsetu_head_list(doc)
    for i, chunk in enumerate(ginza.bunsetu_spans(doc)):
        words.append({
            'text': chunk.text, 'tag': chunk.label_
        })
        for token in chunk.lefts:
            arcs.append({
                'start': bunsetu_head_list.index(token.i),
                'end': i,
                'label': token.dep_,
                'dir': 'left'
            })
        for token in chunk.rights:
            arcs.append({
                'start': i,
                'end': bunsetu_head_list.index(token.i),
                'label': token.dep_,
                'dir': 'right'
            })
    return {'words': words, 'arcs': arcs}

In [14]:
displacy.render(
    to_dependency_data(nlp(t)),
    style='dep',
    jupyter=True,
    manual=True
)

In [ ]:
# for i, chunk in enumerate(ginza.bunsetu_spans(doc)):
#     print(chunk)
#     for token in chunk.lefts:
#         print("\t", token)
#     print()

In [15]:
t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"
bunsetu_ngram = []
n = 2
bunsetu = list( map( str, ginza.bunsetu_spans(doc) ) ) 
for i in range( len(bunsetu)-n+1 ):
    bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )

NameError: name 'doc' is not defined

In [ ]:
bunsetu_ngram

['旅色というスペインの',
 'スペインの赤ワインや、',
 '赤ワインや、プレーンズ・トレインズ・アンド・エリック',
 'プレーンズ・トレインズ・アンド・エリック〜ジャパン・ツアー 2014という',
 '〜ジャパン・ツアー 2014というマドリッドの',
 'マドリッドの白ワインは',
 '白ワインはとても',
 'とても甘口なので',
 '甘口なのでワインが',
 'ワインが苦手な',
 '苦手な方にも',
 '方にもおすすめです。']

In [ ]:
import time
from selenium.webdriver.common.by import By
import spacy
import csv
import copy

In [ ]:
class Seacher:
    def __init__(self) -> None:
        self.query = ""
        self.stop_word = "← ww ・ 「 」 #".split()
        self.nlp =  spacy.load('ja_ginza')

        # self.corpus_path = corpus_path
        # self.name = name

        self.tweet_all = set()
    
    def connect(self, server=None):
        chrome_options = webdriver.ChromeOptions()
        if server:
            command = "http://{0}:4444/wd/hub".format(server)
            self.driver = webdriver.Remote(
                command_executor="http://{0}:4444/wd/hub".format(server),
                options=chrome_options
            )
    
    def query2formated(self, query):
        # splited = query.split()
        return "\""+query+"\""
    
    def search(self, query):
        url = "https://twitter.com/search?f=live&q="

        self.driver.get( url+self.query2formated(query) )

        # 検索結果があるかチェック
        time.sleep(2)
        return self.is_exist_article()
            
    
    def is_exist_article(self):
        # print("start")
        html = self.driver.page_source.encode('utf-8')
        soup = BeautifulSoup(html, 'lxml')
        self.articles = soup.find_all("article")
        # print(self.articles)
        # print("come here")
        if self.articles==[]:
            return False
        else:
            return True
    
    


In [17]:
# def make_bunsetu_ngram(text, n=3):
#     bunsetu_ngram = []
#     bunsetu = list( map( str, ginza.bunsetu_spans(nlp(text)) ) ) 
#     for i in range( len(bunsetu)-n+1 ):
#         bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
#     return bunsetu_ngram

def make_bunsetu_ngram_suf(text, n=3):
    bunsetu_ngram = []
    doc = nlp(text)
    bunsetu = list(map(str, ginza.bunsetu_spans(doc))) 
    phrases = ginza.bunsetu_phrase_spans(doc)
    # print(bunsetu)
    # print(phrases)
    # for i in range( len(bunsetu)-n+1 ):
        # bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
    noun_chunk = [ ]
    for i in range(len(bunsetu)-n+1):
        # print(bunsetu[i],end=" " )
        bunsetu_ngram.append("".join(bunsetu[i:i+n-1])+phrases[i+n-1].orth_)

    # if bunsetu_ngram==[]:
    #     bunsetu_ngram = make_bunsetu_ngram_suf(text, n=2)
    return bunsetu_ngram

t = "私の住んでいる竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送されていません。"
make_bunsetu_ngram_suf(t)

['私の住んでいる竹田城跡',
 '住んでいる竹田城跡では、カミングアウトバラエティ秘密',
 '竹田城跡では、カミングアウトバラエティ秘密のケンミンSHOW',
 'カミングアウトバラエティ秘密のケンミンSHOWがテレビ放送']

In [5]:
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from datatools.analyzer import *

from datatools.maneger import DataManager
from datatools.preproc import Preprocessor

path = "../hand_labeled/"
datalist = ['DCM', 'DIT', 'IRS']
convs = read_conv(path, datalist)

error = "Wrong information"
sys_utt = []
y = []
for conv in convs:
    for ut in conv:
        if ut.is_system() and ut.is_exist_error():
            sys_utt.append(ut.utt)
            if ut.is_error_included(error):
                y.append(1)
            else:
                y.append(0)

In [39]:
seacher = Seacher()
seacher.connect("192.168.1.224")

In [7]:
print("len:", len(y))
print("count({0}): {1}".format(error, y.count(1)))

len: 992
count(Wrong information): 122


In [40]:
q = "赤ワインや、プレーンズ・トレインズ・アンド・エリック"
# seacher.search(q)

In [41]:
y_pred = []
for i, utt in enumerate(sys_utt):
    # print(utt)
    phrase_ngram = make_bunsetu_ngram_suf(utt, n=2)
    # 文節が2つしかない
    if phrase_ngram==[]:
        phrase_ngram = [utt]
    print(i, phrase_ngram)
    error = False
    for q in phrase_ngram:
        if not seacher.search(q):
            print("not found", utt, q)
            error = True
            break
    # break
    # yのチェック
    if error:
        y_pred.append(1)
    else:
        y_pred.append(0)
    print()

0 ['海に行か']

1 ['海は大丈夫']

2 ['海は危険']

3 ['海はいけ']

4 ['わあ！いい']

5 ['ご存知ですい']

6 ['ご存知になら']
not found ご存知になられます ご存知になら

7 ['スイカは実', '実がなっ', 'なってたこと', 'ことあり']
not found スイカは実がなってたことあります スイカは実

8 ['あなたの好き', '好きな映画', '映画について教え']

9 ['映画にはまる']

10 ['映画が楽しみ']

11 ['映画が楽しみ']

12 ['映画館はチケット', 'チケットを買う']

13 ['食事で摂る']

14 ['外でとり']

15 ['外食は二人', '二人で行き']
not found 外食は二人で行きますねぇ 外食は二人

16 ['食事のパートナー', 'パートナーはい', 'いてるんですね。外食', '外食で焼肉', '焼肉に行き']
not found 食事のパートナーはいてるんですね。外食で焼肉に行きますねぇ パートナーはい

17 ['あなた、外食', '外食のとき', 'ときも私', '私を連れ', '連れて行っ']

18 ['夕食は明日', '明日の活力', '活力を生む', '生むから、しっかり', 'しっかり食べ']
not found 夕食は明日の活力を生むから、しっかり食べてね。 生むから、しっかり

19 ['野菜はカロリー', 'カロリーが低い']

20 ['カロリーは欲しい']

21 ['ラーメンが食べ']

22 ['ラーメンは美味しい']

23 ['麺が美味しい']

24 ['パスタが好き']

25 ['パスタはクリーム系', 'クリーム系が好き']

26 ['海は楽しい']

27 ['海はサイコー']

28 ['海はキレー']

29 ['魚はおい', 'おいんですか？？海', '海は素晴らしい']

30 ['海は素晴らしい']

31 ['台風が凄い']

32 ['熱中症はいい']
not found 熱中症はいいですね 熱中症はいい

33 ['熱中症は大丈夫']

34 ['手洗いうがいで予防', '予防が大切']

35 ['予防は手洗い', '手洗いが基本']

36 ['予防は難しい

In [46]:
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y, y_pred=y_pred))
print('precision = ', precision_score(y_true=y, y_pred=y_pred))
print('recall = ', recall_score(y_true=y, y_pred=y_pred))
print('f1 score = ', f1_score(y_true=y, y_pred=y_pred))

confusion matrix = 
 [[519 454]
 [  2 374]]
accuracy =  0.6619718309859155
precision =  0.45169082125603865
recall =  0.9946808510638298
f1 score =  0.6212624584717609


In [47]:

# for x, p in zip(sys_utt, y):
#     if p==1:
#         print(x)

In [44]:
y.count(1)

376

In [45]:
def make_bunsetu_ngram_suf(text, n=3):
    bunsetu_ngram = []
    doc = nlp(text)
    bunsetu = list(map(str, ginza.bunsetu_spans(doc))) 
    phrases = ginza.bunsetu_phrase_spans(doc)
    # print(bunsetu)
    # print(phrases)
    # for i in range( len(bunsetu)-n+1 ):
        # bunsetu_ngram.append( "".join(bunsetu[i:i+n]) )
    noun_chunk = [ ]
    for i in range(len(bunsetu)-n+1):
        # print(bunsetu[i],end=" " )
        bunsetu_ngram.append("".join(bunsetu[i:i+n-1])+phrases[i+n-1].lemma_)

    if bunsetu_ngram==[]:
        bunsetu_ngram = make_bunsetu_ngram_suf(text, n=2)
    return bunsetu_ngram
tt = "腰も治りますよね～"
make_bunsetu_ngram_suf(tt)

['腰も治る']

In [49]:
data_path = "./wrong/"
data_name = "wrong_tw_res.pickle"
dataM = DataManager(data_path)

In [50]:
dataM.save_data(data_name, [y, y_pred])

success save : ./wrong/wrong_tw_res.pickle
